In [43]:
from sklearn import neighbors
from sklearn.decomposition import PCA
from arch.bootstrap import StationaryBootstrap
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import math
import sys
import random
from itertools import chain

In [99]:
#read training and cross validation set
total = pd.read_csv("time_space.csv")
n = total.shape[0]
num_tract = 1758

test = pd.read_csv("test_data.csv")
m = test.shape[0]
test = test.drop(test.columns[0], axis = 1) #delete index
test_geoid = test['GEOID']
test = test.drop(test.columns[0], axis = 1) #delete GEOID
for i in range(m):
    now = dt.datetime.strptime(test.loc[i, 'MP_DATE'],'%Y-%m-%d').date()
    test.loc[i, 'MP_DATE'] = (now - origin).days/30.0
    

total = total.drop(total.columns[0], axis = 1) #delete index

tract_id = total['GEOID']
total = total.drop(total.columns[0], axis = 1) #delete tract_geoid
    
mp_date = total['MP_DATE']
origin = dt.datetime.strptime(total.loc[0, 'MP_DATE'],'%Y-%m-%d').date()
for i in range(n):
    now = dt.datetime.strptime(total.loc[i, 'MP_DATE'],'%Y-%m-%d').date()
    total.loc[i, 'MP_DATE'] = (now - origin).days/30.0

values = total['values']
total = total.drop(total.columns[3], axis = 1)

In [3]:
for blocksize in range(25):
    print("Current Block Size: "+str(blocksize))
    
    #get 10 block bootstrapping samples
    for j in range(num_tract):
        sub_data = total.iloc[63*j:63*(j+1)]
        bs = StationaryBootstrap(blocksize, sub_data)
        if j == 0:
            train_lst = [data[0][0] for data in bs.bootstrap(10)]
        else:
            i = 0
            for data in bs.bootstrap(10):
                train_lst[i] = train_lst[i].append(data[0][0])
                i += 1
    print("Finish Bootstrapping")

            
    #get unselected samples as my cross validation
    ind_all = set(total.index)
    ind_diff = []
    possible_oob = set(chain.from_iterable([list(range(63*k+24,63*(k+1)+1)) for k in range(1758)]))
    d_oob_learner = dict()
    d_oob_cv = dict()

    for i in range(10):
        ind_unselected = list(possible_oob.intersection(ind_all - set(train_lst[i].index)))
        print(len(ind_unselected))
        for item in ind_unselected:
            try:
                d_oob_learner[item] += [i]
            except:
                d_oob_learner[item] = [i]
        d_oob_cv[i] = ind_unselected
    print("Finish collecting feasible OOB for CV and prediction")
    
    
    #find best k for each bootstrapping sample --> model parameter
    best_k_lst = []
    best_mae_lst = []
    symbol_lst = []
    y_ = []
    for i in range(10):
        train_y = train_lst[i]['dv'] #get dependent variable
        train_x = train_lst[i].drop(train_lst[i].columns[3], axis = 1) #delete dependent variables

        best_mae = sys.maxsize
        for k in range(1, 32, 5):
            knn = neighbors.KNeighborsRegressor(k, weights ='distance')
            y_ = []
            y_mean = 0
            y_symbol = 0
            cv_times = range(51,62)
            for cv_time in cv_times:
                train_id = list(chain.from_iterable([list(range(63*k,(63*k+cv_time))) for k in range(num_tract)]))
                test_id = [63*k+cv_time for k in range(num_tract)]
            #print(test_id)
                y_predict = knn.fit(train_x.iloc[train_id], train_y.iloc[train_id]).predict(train_x.iloc[test_id])
                y_ += [y_predict]
                mae = np.mean(abs(y_predict - train_y.iloc[test_id]))
                y_mean += mae
                y_symbol += sum(y_predict*train_y.iloc[test_id]>=0)/len(y_predict)
            y_mean = y_mean / len(cv_times)
            y_symbol = y_symbol / len(cv_times)
            if y_mean < best_mae:
                best_mae = y_mean
                best_k = k
                symbol = y_symbol
                print((i, k, best_mae))
        best_mae_lst.append(best_mae)
        best_k_lst.append(best_k)
        symbol_lst.append(symbol)



In [80]:
blocksize = 12
#get 10 block bootstrapping samples
for j in range(num_tract):
    sub_data = total.iloc[63*j:63*(j+1)]
    bs = StationaryBootstrap(blocksize, sub_data)
    if j == 0:
        train_lst = [data[0][0] for data in bs.bootstrap(10)]
    else:
        i = 0
        for data in bs.bootstrap(10):
            train_lst[i] = train_lst[i].append(data[0][0])
            i += 1
print("Finish Bootstrapping")

            
#get unselected samples as my cross validation
ind_all = set(total.index)
ind_diff = []
possible_oob = set(chain.from_iterable([list(range(63*k+24,63*(k+1)+1)) for k in range(1758)]))
d_oob_learner = dict()
d_oob_cv = dict()

for i in range(10):
    ind_unselected = list(possible_oob.intersection(ind_all - set(train_lst[i].index)))
    print(len(ind_unselected))
    for item in ind_unselected:
        try:
            d_oob_learner[item] += [i]
        except:
            d_oob_learner[item] = [i]
    d_oob_cv[i] = ind_unselected
print("Finish collecting feasible OOB for CV and prediction")
    


Finish Bootstrapping
20476
21251
21226
21094
20826
20674
21109
20762
20549
20726
Finish collecting feasible OOB for CV and prediction


In [108]:
#find best k for each bootstrapping sample --> model parameter
best_k_lst = []
best_mae_lst = []
symbol_lst = []
y_ = []
for i in range(10):
    train_y = train_lst[i]['dv'] #get dependent variable
    train_x = train_lst[i].drop(train_lst[i].columns[3], axis = 1) #delete dependent variables

    best_mae = sys.maxsize
    for k in range(1, 32, 5):
        knn = neighbors.KNeighborsRegressor(k, weights ='distance')
        y_ = []
        y_mean = 0
        y_symbol = 0
        cv_times = range(51,62)
        for cv_time in cv_times:
            train_id = list(chain.from_iterable([list(range(63*k,(63*k+cv_time))) for k in range(num_tract)]))
            test_id = [63*k+cv_time for k in range(num_tract)]
            #print(test_id)
            y_predict = knn.fit(train_x.iloc[train_id], train_y.iloc[train_id]).predict(train_x.iloc[test_id])
            y_ += [y_predict]
            mae = np.mean(abs(y_predict - train_y.iloc[test_id]))
            y_mean += mae
            y_symbol += sum(y_predict*train_y.iloc[test_id]>=0)/len(y_predict)
        y_mean = y_mean / len(cv_times)
        y_symbol = y_symbol / len(cv_times)
        if y_mean < best_mae:
            best_mae = y_mean
            best_k = k
            symbol = y_symbol
            print((i, k, best_mae))
    best_mae_lst.append(best_mae)
    best_k_lst.append(best_k)
    symbol_lst.append(symbol)

(0, 1, 1.3070919906956553)
(1, 1, 2.2871430531392467)
(1, 6, 1.9018020232025092)
(2, 1, 1.9237542514846426)
(2, 6, 1.8901346853445835)
(3, 1, 2.5282974141361536)
(4, 1, 2.4005230923152543)
(5, 1, 1.9937114036356918)
(6, 1, 4.722418783507211)
(6, 6, 4.025679660454961)
(7, 1, 1.5800129788458541)
(8, 1, 1.4488189429000804)
(9, 1, 1.8202846204410854)


In [6]:
#get predictions
y_pre_lst = []
for i in range(10):
    train_y = train_lst[i]['dv'] #get dependent variable
    train_x = train_lst[i].drop(train_lst[i].columns[3], axis = 1) #delete dependent variables
    knn = neighbors.KNeighborsRegressor(best_k_lst[i], weights ='distance')
    y_pre = knn.fit(train_x, train_y).predict(test)
    y_pre_lst.append(y_pre)


In [100]:
best_k_lst

[1, 31, 11, 31, 16, 31, 31, 31, 21, 6]

In [109]:
symbol_lst

[0.96080256489812799,
 0.9571310373358155,
 0.95454545454545459,
 0.96493949736270568,
 0.96261247285138074,
 0.9635432826559106,
 0.95020167545764833,
 0.96287103113041683,
 0.96318130106526023,
 0.96178508635846538]

In [105]:
np.mean(best_mae_lst)

2.0896356811971826

In [7]:
#get predictions
y_pre_lst = []
for i in range(10):
    for 
    train_y = train_lst[i]['dv'] #get dependent variable
    train_x = train_lst[i].drop(train_lst[i].columns[3], axis = 1) #delete dependent variables
    knn = neighbors.KNeighborsRegressor(best_k_lst[i], weights ='distance')
    y_pre = knn.fit(train_x, train_y).predict(test)
    y_pre_lst.append(y_pre)

In [8]:
y_pre_mat = np.array(y_pre_lst)
y_final_pre = np.mean(y_pre_mat, 0)

In [11]:
for i in range(1758):
    filename = "%d.png" % i
    plt.figure(figsize=(8,6), dpi=60)
    plt.plot(range(12), y_final_pre[i*12:(i+1)*12], 'k')
    plt.savefig(filename, dpi = 60)

E:\Anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


SystemError: <built-in method write of _io.BufferedWriter object at 0x000001E0E14E60F8> returned a result with an error set